In [1]:
from tensorflow.keras.datasets import mnist, cifar10
from thirdai import bolt, dataset
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
(orig_xtrain, orig_ytrain), (orig_xtest, orig_ytest) = cifar10.load_data()

In [3]:
num_train_samples, height, width, channels = orig_xtrain.shape
input_image_size = height * width * channels

n_classes = 10

In [4]:
def get_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
    )

    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow(
        orig_xtrain, orig_ytrain,
        shuffle=True,
        batch_size=len(orig_xtrain),  # number of train samples
    )

    test_generator = test_datagen.flow(
        orig_xtest, orig_ytest,
        shuffle=True,
        batch_size=len(orig_xtest),  # number of test samples
    )

    return train_generator, test_generator

In [5]:
def process_data(train_gen, test_gen):
    modified_xtrain, modified_ytrain = train_gen.next()
    modified_xtest, modified_ytest = test_gen.next()

    xtrain = np.reshape(orig_xtrain, (50000, input_image_size))
    ytrain = np.reshape(orig_ytrain, (50000,))
    xtest = np.reshape(orig_xtest, (10000, input_image_size))
    ytest = np.reshape(orig_ytest, (10000,))

    xtrain = xtrain.astype("float32")
    ytrain = ytrain.astype("uint32")
    xtest = xtest.astype("float32")
    ytest = ytest.astype("uint32")

    batch_size = 256

    xtrain = dataset.from_numpy(xtrain, batch_size=batch_size)
    ytrain = dataset.from_numpy(ytrain, batch_size=batch_size)
    xtest = dataset.from_numpy(xtest, batch_size=batch_size)
    ytest = dataset.from_numpy(ytest, batch_size=batch_size)

    return xtrain, ytrain, xtest, ytest

In [6]:
from thirdai import bolt

input_layer = bolt.nn.Input(dim=input_image_size)

hidden_layer = bolt.nn.FullyConnected(
    dim=1000,
    sparsity=0.3,
    activation="relu",
)(input_layer)
output_layer = bolt.nn.FullyConnected(dim=n_classes, activation="softmax")(
    hidden_layer
)

model = bolt.nn.Model(inputs=[input_layer], output=output_layer)

model.compile(bolt.nn.losses.CategoricalCrossEntropy())

epochs = 5

for e in range(epochs):
    train_gen, test_gen = get_data_generators()

    xtrain, ytrain, xtest, ytest = process_data(train_gen, test_gen)

    train_cfg = bolt.TrainConfig(epochs=1, learning_rate=0.001).with_metrics(["categorical_accuracy"])
    model.train(xtrain, ytrain, train_cfg)

    eval_cfg = bolt.EvalConfig().with_metrics(["categorical_accuracy"])
    metrics = model.evaluate(xtest, ytest, eval_cfg)


======================= Bolt Model =======================
input_1 (Input): dim=3072
input_1 -> fc_1 (FullyConnected): dim=1000, sparsity=0.3, act_func=ReLU
fc_1 -> fc_2 (FullyConnected): dim=10, sparsity=1, act_func=Softmax

train epoch 0:

[==================================================] 100%

train | epoch 0 | updates 196 | {categorical_accuracy: 0.13684} | batches 196 | time 22s | complete

test:

[==================================================] 100%

predict | epoch 1 | updates 196 | {categorical_accuracy: 0.1919} | batches 40 | time 27886ms

train epoch 1:

[==================================================] 100%

train | epoch 1 | updates 392 | {categorical_accuracy: 0.15722} | batches 196 | time 21s | complete

test:

[==================================================] 100%

predict | epoch 2 | updates 392 | {categorical_accuracy: 0.2274} | batches 40 | time 28036ms

train epoch 2:

[==================================================] 100%

train | epoch 2 | updates 